In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np
import math
from scipy import stats
import os
import time 
from datetime import datetime
from itertools import chain
import subprocess
import gc

In [ ]:
import importlib
import SICAK_SPECIFIC_T_TEST as sicak

# Reload the module to apply changes
importlib.reload(sicak)

# Import the updated functions
import SICAK_SPECIFIC_T_TEST as sicak

In [ ]:
import importlib
import _ChaCha20_source

# Reload the module to apply changes
importlib.reload(_ChaCha20_source)

# Import the updated functions
from _ChaCha20_source import Chacha20, Chacha20Keystream

# Helper functions to load data

In [ ]:
CHUNK_SIZE = None
CHUNKS_CNT = None
LAST_CHUNK_SIZE = None

TRACE_CNT = None
TRACE_LEN = None
TRACE_RANDOM_CNT = None

NONCES = None
NONCE_LEN_BYTES = 12  # Length of each nonce in bytes
NONCE_LEN_32BIT = NONCE_LEN_BYTES // 4  # Number of 32-bit integers per nonce

In [ ]:
def read_info(folder):
    """Reads the info.txt file to get the number of traces, chunk size, and trace length."""
    global CHUNK_SIZE, LAST_CHUNK_SIZE, CHUNKS_CNT
    with open(f"{folder}/info.txt", 'r') as file:
        global TRACE_CNT
        global TRACE_LEN
        global TRACE_RANDOM_CNT
        TRACE_CNT = int(file.readline())
        TRACE_RANDOM_CNT = TRACE_CNT
        CHUNK_SIZE = int(file.readline())
        TRACE_LEN = int(file.readline())
        
    CHUNKS_CNT = math.ceil(TRACE_CNT / CHUNK_SIZE)
    LAST_CHUNK_SIZE = TRACE_CNT - (CHUNKS_CNT - 1)* CHUNK_SIZE
    print(f"TRACE_CNT = {TRACE_CNT}")   
    print(f"CHUNK_SIZE = {CHUNK_SIZE}")   
    print(f"LAST_CHUNK_SIZE = {LAST_CHUNK_SIZE}")   
    print(f"CHUNKS_CNT = {CHUNKS_CNT}")   
    print(f"TRACE_RANDOM_CNT = {TRACE_RANDOM_CNT}")   
    print(f"TRACE_LEN = {TRACE_LEN}")

In [ ]:
def read_nonces(folder):
    """Reads the nonces from the binary files in the specified folder."""
    global NONCES, CHUNK_SIZE, LAST_CHUNK_SIZE, CHUNKS_CNT, TRACE_CNT
    
    nonces_list = []
    for chunk_index in range(CHUNKS_CNT):
        print(f"Processing CHUNK - {chunk_index}")
        chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
        chunk_file = os.path.join(chunk_folder, "nonces_random.bin")
        
        if os.path.exists(chunk_file):
            with open(chunk_file, 'rb') as file:
                byte_array = file.read()
            
            if chunk_index != CHUNKS_CNT - 1:
                chunk = np.frombuffer(byte_array, dtype=np.uint32).reshape((CHUNK_SIZE, NONCE_LEN_32BIT))
            else:
                chunk = np.frombuffer(byte_array, dtype=np.uint32).reshape((LAST_CHUNK_SIZE, NONCE_LEN_32BIT))
            nonces_list.append(chunk)
        else:
            print(f"Chunk file {chunk_file} does not exist.")
    
    # Concatenate all chunks
    NONCES = np.vstack(nonces_list)

In [ ]:
def compress_traces_into_one_file(folder):
    """Compresses all traces from multiple chunk files into a single binary file within the folder provided."""
    output_file = os.path.join(folder, "traces_random_all.bin")

    # Open the output file in write-binary mode
    with open(output_file, 'wb') as outfile:
        for chunk_index in range(CHUNKS_CNT):
            chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
            chunk_file = os.path.join(chunk_folder, "traces_random.bin")

            print(f"Processing CHUNK - {chunk_index}")

            # Check if the chunk file exists
            if os.path.exists(chunk_file):
                # Open the chunk file in read-binary mode
                with open(chunk_file, 'rb') as infile:
                    # Read the contents of the chunk file
                    chunk_data = infile.read()
                    # Write the contents to the output file
                    outfile.write(chunk_data)
            else:
                print(f"Chunk file {chunk_file} does not exist.")

In [ ]:
sigma_const = b"expand 32-byte k"
# Convert the byte string to an array of 32-bit unsigned integers
sigma_const = np.frombuffer(sigma_const, dtype=np.uint32)

# Helper functions to split traces, calculate intermediate values

In [ ]:
S1_ARRAY = None
LF0 = 0
LF1 = 1
def leftRotate16(n):
    return ((n << 16) | (n >> 16)) & 0xFFFFFFFF

def calculate_intermediate_value(function_index, byte_index, key, nonce=None, sigma=None, previous_intermediate_value=None, recovered_key=None):
    """
    Calculate the intermediate value based on the function index and byte index.
    The function index determines which calculation to perform, and the byte index specifies which byte of the result to return.

    Parameters:
    - function_index (int): The index of the function to use (0 or 1).
    - byte_index (int): The index of the byte to return (0 to 3).
    - key (int): The key value (32-bit unsigned integer).
    - nonce (int, optional): The nonce value (32-bit unsigned integer) for function index 0.
    - sigma (int, optional): The sigma value (32-bit unsigned integer) for function index 0.
    - previous_intermediate_value (int, optional): The previous intermediate value (32-bit unsigned integer) for function index 1.
    - recovered_key (int, optional): The recovered key value (32-bit unsigned integer) for function index 1.

    Returns:
    - int: The calculated intermediate value (8-bit unsigned integer).
    """
    shift = byte_index * 8
    if byte_index < 0 or byte_index > 3:
        raise Exception("Invalid byte index")
    
    if function_index == 0:
        # Usage of basic leakage function: nonce ^ (key + sigma)
        if sigma is None or nonce is None:
            raise Exception("Sigma value is required for this calculation")
        return ((nonce ^ (key + sigma)) >> shift) & 0xFF
    elif function_index == 1:
        # Usage of advanced leakage function: (key + leftRotate16(previous_intermediate_value)) ^ recovered_key
        # The leftRotate16 function is used to rotate the previous intermediate value by 16 bits
        # The previous intermediate value is calculated using the basic leakage function
        if previous_intermediate_value is None or recovered_key is None:
            raise Exception("Previous intermediate value and recovered key are required for this calculation")
        rotated_value = leftRotate16(previous_intermediate_value)
        return ((key + rotated_value) ^ recovered_key) >> shift & 0xFF
    else:
        raise Exception("Invalid LF index")
        
def calculate_state_s1():
    """
    Calculate the S[1] state matrix for the given traces and nonces.
    The S[1] state matrix is a 4x4 matrix of 32-bit values, where each value is derived from the keystream generated by the ChaCha20 algorithm.
    """
    # Calculate the S[1] - TRACE_CNT * State matrix (4x4 32bit values)
    s1_array = np.zeros((TRACE_CNT, 4 * 4), dtype=np.uint32)
    # Iterate over each trace
    for i in range(TRACE_CNT):
        # Get the nonce for the current trace
        nonce = NONCES[i]
        # Generate the keystream using the correct key and nonce
        keystream = Chacha20Keystream(CORRECT_KEY, nonce, rounds=1)

        # Reshape the keystream into a 4x4 matrix and store it in the S[1] matrix
        s1_array[i] = np.array(keystream)
    return s1_array

# ORIGINAL VERSION, WHICH DOES NOT SUPPORT LA FOR STATE S1[0] and S1[12], ONLY SUPPORTS S1[4] and S1[8]
def calculate_intermediate_values(byte_index):
    """
    Calculate the intermediate values for a specific byte index.
    The byte index determines which byte of the result to return.

    Parameters:
    - byte_index (int): The index of the byte to return (0 to 31).

    Returns:
    - np.ndarray: The calculated intermediate values (8-bit unsigned integers).
    """
    global TRACE_CNT, LF0, LF1, NONCES, sigma_const, S1_ARRAY, CORRECT_KEY
    intermediate_values = np.zeros(TRACE_CNT, dtype=np.uint8)
    if byte_index in range(0,4):
        # Handle byte_index values from 0 to 3
        if S1_ARRAY is None:
            raise Exception("S1 array is required for this calculation")
        # Calculate the intermediate values
        for i in range(TRACE_CNT):
            state_s1_key = S1_ARRAY[i][4]
            state_s1_sigma = S1_ARRAY[i][3]
            state_s1_nonce = S1_ARRAY[i][14]
            intermediate_values[i] = calculate_intermediate_value(LF0, byte_index % 4, state_s1_key, nonce=state_s1_nonce, sigma=state_s1_sigma)
    elif byte_index in range(4,16):
        # Handle byte_index values from 4 to 15
        col_index = byte_index // 4
        key = CORRECT_KEY[col_index]
        sigma = sigma_const[col_index]
        for i in range(TRACE_CNT):
            nonce = NONCES[i][col_index - 1]
            intermediate_values[i] = calculate_intermediate_value(LF0, byte_index % 4, key, nonce=nonce, sigma=sigma)
    elif byte_index in range(16,20):
        # Handle byte_index values from 16 to 19
        if S1_ARRAY is None:
            raise Exception("S1_ARRAY is required for this calculation")
        # Calculate the intermediate values
        for i in range(TRACE_CNT):
            state_s1_sigma    = S1_ARRAY[i][2]
            state_s1_prev_key = S1_ARRAY[i][7]
            state_s1_key      = S1_ARRAY[i][8]
            state_s1_nonce    = S1_ARRAY[i][13]
            prev_i_v = state_s1_nonce ^ ((state_s1_prev_key + state_s1_sigma) & 0xFFFFFFFF)
            intermediate_values[i] = calculate_intermediate_value(LF1, byte_index % 4, state_s1_key, nonce=None, sigma=None, previous_intermediate_value=prev_i_v, recovered_key=state_s1_prev_key)
    elif byte_index in range(20,32):
        col_index = (byte_index - 16) // 4
        prev_key = CORRECT_KEY[col_index]
        sigma = sigma_const[col_index]
        key = CORRECT_KEY[col_index + 4]

        for i in range(TRACE_CNT):
            prev_i_v = NONCES[i][col_index - 1] ^ ((prev_key + sigma) & 0xFFFFFFFF)
            intermediate_values[i] = calculate_intermediate_value(LF1, byte_index % 4, key, nonce=None, sigma=None, previous_intermediate_value=prev_i_v, recovered_key=prev_key)
    else:
        raise Exception("Not implemented")
    
    return intermediate_values

# ADDITIONAL VERSION, WHICH DOES NOT SUPPORT LA FOR KEY BYTES, BUT SUPPORTS LA FOR STATE S1[0], S1[12], S1[4], and S1[8]
def calculate_intermediate_values_S1_column(word_index, byte_index):
    """
    Calculate the intermediate values for a specific column of the S[1] state matrix.
    The column is specified by the word index and byte index.
    The word index determines which column to use (0, 4, 8, or 12), and the byte index specifies which byte of the result to return.

    Parameters:
    - word_index (int): The index of the word to use (0, 4, 8, or 12).
    - byte_index (int): The index of the byte to return (0 to 3).

    Returns:
    - np.ndarray: The calculated intermediate values (8-bit unsigned integers).
    """

    global S1_ARRAY, TRACE_CNT, LF0, LF1

    if S1_ARRAY is None:
            raise Exception("S1 array is required for this calculation")

    if word_index not in [0, 4, 8, 12]:
        raise Exception("Invalid word index. Must be one of [0, 4, 8, 12]")
    
    if byte_index not in range(0, 4):
        raise Exception("Invalid byte index. Must be in range [0, 4)")
    
    intermediate_values = np.zeros(TRACE_CNT, dtype=np.uint8)        
    if word_index == 0:
        for i in range(TRACE_CNT):
            Sx = S1_ARRAY[i][0]
            Sy = S1_ARRAY[i][5]
            Sz = S1_ARRAY[i][15]
            intermediate_values[i] = calculate_intermediate_value(LF0, byte_index, key=Sy, nonce=Sz, sigma=Sx)
    elif word_index == 4:
        for i in range(TRACE_CNT):
            Sx = S1_ARRAY[i][3]
            Sy = S1_ARRAY[i][4]
            Sz = S1_ARRAY[i][14]
            intermediate_values[i] = calculate_intermediate_value(LF0, byte_index, key=Sy, nonce=Sz, sigma=Sx)

    elif word_index == 8:
        for i in range(TRACE_CNT):
            Sx = S1_ARRAY[i][2]
            Sy = S1_ARRAY[i][7]
            Sz = S1_ARRAY[i][13]
            Sk = S1_ARRAY[i][8]
            prev_i_v = Sz ^ ((Sy + Sx) & 0xFFFFFFFF)
            intermediate_values[i] = calculate_intermediate_value(LF1, byte_index, key=Sk, nonce=None, sigma=None, previous_intermediate_value=prev_i_v, recovered_key=Sy)
    elif word_index == 12:
        for i in range(TRACE_CNT):
            Sx = S1_ARRAY[i][1]
            Sy = S1_ARRAY[i][6]
            Sz = S1_ARRAY[i][12]
            intermediate_values[i] = calculate_intermediate_value(LF0, byte_index, key=Sy, nonce=Sz, sigma=Sx)
    else:
        raise Exception("Not implemented")
    
    return intermediate_values

# Define a named tuple to hold the t-test results
from collections import namedtuple
TTestResult = namedtuple('TTestResult', ['statistic', 'pvalue'])

In [ ]:
# ORIGINAL VERSION, WHICH DOES NOT SUPPORT LA FOR STATE S1[0] and S1[12], ONLY SUPPORTS S1[4] and S1[8]
def sort_traces_and_split(output_folder, input_file = "traces_random_all.bin", chunks = 10, byte_index=0, byte_value=0x00):
    """
    Sort traces based on the intermediate value at a specific byte index.

    This function sorts traces into chunks paired files, where each chunk contains traces that either match or do not match the specified byte value at the given byte index.
    The traces are read from the input file, and the sorted traces are saved into separate files in the specified output folder.

    Args:
        output_folder (str): The folder where the sorted traces will be saved.
        input_file (str): The input file containing the traces (default is "traces_random_all.bin").
        chunks (int): The number of chunks, into which is the input_file divided (default is 10).
        byte_index (int): The byte index to check in the intermediate value.
        byte_value (int): The byte value to compare against (default is 0x00).

    Returns:
        None
    """
    global TRACE_LEN, TRACE_CNT
    # Start timing
    start_time = time.time()

    print(f".byte index = {byte_index}, byte value = {hex(byte_value)}")

    # Precalculate intermediate values
    intermediate_values = calculate_intermediate_values(byte_index)
    print(f"..intermediate values calculated - size {len(intermediate_values)}")

    # Initialize output folder
    os.makedirs(output_folder, exist_ok=True)

    # Initialize output files for 10 chunks
    equal_files = [
        open(os.path.join(output_folder, f"traces_equal_{byte_index:02x}_{byte_value:02x}-{chunk}.bin"), 'wb')
        for chunk in range(chunks)
    ]
    not_equal_files = [
        open(os.path.join(output_folder, f"traces_not_equal_{byte_index:02x}_{byte_value:02x}-{chunk}.bin"), 'wb')
        for chunk in range(chunks)
    ]

    # Open the input file
    with open(input_file, 'rb') as infile:
        # Iterate over the traces and sort them directly into chunks
        for i in range(TRACE_CNT):
            trace = np.frombuffer(infile.read(TRACE_LEN * np.dtype(np.uint16).itemsize), dtype=np.uint16)
            chunk_index = (i * chunks) // TRACE_CNT  # Determine the chunk index (0-9)

            if intermediate_values[i] == byte_value:
                equal_files[chunk_index].write(trace.tobytes())
            else:
                not_equal_files[chunk_index].write(trace.tobytes())

    # Close all output files
    for f in equal_files + not_equal_files:
        f.close()

    # End timing
    end_time = time.time()
    duration_seconds = end_time - start_time
    print(f"Traces sorted and split into {chunks} parts. Duration: {duration_seconds:.2f} seconds")

# ADDITIONAL VERSION, WHICH DOES NOT SUPPORT LA FOR KEY BYTES, BUT SUPPORTS LA FOR STATE S1[0], S1[12], S1[4], and S1[8]
def sort_traces_and_split_s1(output_folder, input_file = "traces_random_all.bin", chunks = 10, word_index=0, byte_index=0, byte_value=0x00):
    """
    Sort traces based on the intermediate value at a specific word index and byte index.

    NOTE: This function is specifically designed ad-hoc for the first column of S[1] state matrix, which is a 4x4 matrix of 32-bit values.

    This function sorts traces into chunks paired files, where each chunk contains traces that either match or do not match the specified byte value at the given byte index.
    The traces are read from the input file, and the sorted traces are saved into separate files in the specified output folder.

    Args:
        output_folder (str): The folder where the sorted traces will be saved.
        input_file (str): The input file containing the traces (default is "traces_random_all.bin").
        chunks (int): The number of chunks, into which is the input_file divided (default is 10).
        word_index (int): The word index to check in the intermediate value (0, 4, 8, or 12).
        byte_index (int): The byte index to check in the intermediate value. (0 to 3).
        byte_value (int): The byte value to compare against (default is 0x00).

    Returns:
        None
    """
    global TRACE_LEN, TRACE_CNT
    # Start timing
    start_time = time.time()

    print(f".word_index = {word_index}, byte index = {byte_index}, byte value = {hex(byte_value)}")

    # Precalculate intermediate values
    intermediate_values = calculate_intermediate_values_S1_column(word_index, byte_index)
    print(f"..intermediate values calculated - size {len(intermediate_values)}")

    # Initialize output folder
    os.makedirs(output_folder, exist_ok=True)

    # Initialize output files for 10 chunks
    equal_files = [
        open(os.path.join(output_folder, f"traces_equal_{word_index:02x}_{byte_index:02x}_{byte_value:02x}-{chunk}.bin"), 'wb')
        for chunk in range(chunks)
    ]
    not_equal_files = [
        open(os.path.join(output_folder, f"traces_not_equal_{word_index:02x}_{byte_index:02x}_{byte_value:02x}-{chunk}.bin"), 'wb')
        for chunk in range(chunks)
    ]

    # Open the input file
    with open(input_file, 'rb') as infile:
        # Iterate over the traces and sort them directly into chunks
        for i in range(TRACE_CNT):
            trace = np.frombuffer(infile.read(TRACE_LEN * np.dtype(np.uint16).itemsize), dtype=np.uint16)
            chunk_index = (i * chunks) // TRACE_CNT  # Determine the chunk index (0-9)

            if intermediate_values[i] == byte_value:
                equal_files[chunk_index].write(trace.tobytes())
            else:
                not_equal_files[chunk_index].write(trace.tobytes())

    # Close all output files
    for f in equal_files + not_equal_files:
        f.close()

    # End timing
    end_time = time.time()
    duration_seconds = end_time - start_time
    print(f"Traces sorted and split into {chunks} parts. Duration: {duration_seconds:.2f} seconds")
    
def clean_up_trace_files(folder, equal_prefix, not_equal_prefix):
    """
    Remove files with specified prefixes (e.g., traces_equal and traces_not_equal) from the given folder.

    Parameters:
        folder (str): Path to the folder containing the trace files.
        equal_prefix (str): Prefix for equal trace files (e.g., "traces_equal").
        not_equal_prefix (str): Prefix for not-equal trace files (e.g., "traces_not_equal").
    """
    # Iterate over the files in the folder
    for filename in os.listdir(folder):
        # Check if the file starts with the specified prefixes
        if filename.startswith(equal_prefix) or filename.startswith(not_equal_prefix):
            file_path = os.path.join(folder, filename)
            try:
                os.remove(file_path)
                print(f"Removed file: {file_path}")
            except OSError as e:
                print(f"Error removing file {file_path}: {e}")

# T-test calculation using SICAK

In [ ]:
equal_prefix = "traces_equal"
not_equal_prefix = "traces_not_equal"

## Specific t-test for STM - Key 1

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-STM"
output_folder = "_LA-STM-1-TEST"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for byte_pos in chain(range(32)): #byte pos 0 - 31
    for byte_value in range(0x100): #0x00 - 0xff
        # Print the current time
        print(f"Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
        
        sort_traces_and_split(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                      chunks = 10, byte_index=byte_pos, byte_value=byte_value)
        
        
        sicak.perform_t_test(folder=output_folder, 
                             trace_len=TRACE_LEN, 
                             file_count=10,
                             output_file_name=f"t_stats_{byte_pos}_{byte_value:02x}.npy", 
                             file_equal_prefix=f"traces_equal_{byte_pos:02x}_{byte_value:02x}",
                             file_not_equal_prefix=f"traces_not_equal_{byte_pos:02x}_{byte_value:02x}",
                             stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
        
        clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for STM - Key 2

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-STM-2"
output_folder = "_LA-STM-2"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for byte_pos in chain(range(0, 32)): #byte pos 0 - 31
    for byte_value in range(0x100): #0x00 - 0xff
        # Print the current time
        print(f"Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
        
        sort_traces_and_split(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                      chunks = 10, byte_index=byte_pos, byte_value=byte_value)
        
        
        sicak.perform_t_test(folder=output_folder, 
                             trace_len=TRACE_LEN, 
                             file_count=10,
                             output_file_name=f"t_stats_{byte_pos}_{byte_value:02x}.npy", 
                             file_equal_prefix=f"traces_equal_{byte_pos:02x}_{byte_value:02x}",
                             file_not_equal_prefix=f"traces_not_equal_{byte_pos:02x}_{byte_value:02x}",
                             stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
        
        clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for XMEGA - Key 1

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-XMEGA"
output_folder = "_LA-XMEGA-1"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for byte_pos in chain(range(0, 32)): #byte pos 0 - 31
    for byte_value in range(0x100): #0x00 - 0xff
        # Print the current time
        print(f"Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
        
        sort_traces_and_split(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                      chunks = 10, byte_index=byte_pos, byte_value=byte_value)
        
        
        sicak.perform_t_test(folder=output_folder, 
                             trace_len=TRACE_LEN, 
                             file_count=10,
                             output_file_name=f"t_stats_{byte_pos}_{byte_value:02x}.npy", 
                             file_equal_prefix=f"traces_equal_{byte_pos:02x}_{byte_value:02x}",
                             file_not_equal_prefix=f"traces_not_equal_{byte_pos:02x}_{byte_value:02x}",
                             stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
        
        clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for XMEGA - Key 2

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-XMEGA-2"
output_folder = "_LA-XMEGA-2"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for byte_pos in chain(range(0, 32)): #byte pos 0 - 31
    for byte_value in range(0x100): #0x00 - 0xff
        # Print the current time
        print(f"Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
        
        sort_traces_and_split(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                      chunks = 10, byte_index=byte_pos, byte_value=byte_value)
        
        
        sicak.perform_t_test(folder=output_folder, 
                             trace_len=TRACE_LEN, 
                             file_count=10,
                             output_file_name=f"t_stats_{byte_pos}_{byte_value:02x}.npy", 
                             file_equal_prefix=f"traces_equal_{byte_pos:02x}_{byte_value:02x}",
                             file_not_equal_prefix=f"traces_not_equal_{byte_pos:02x}_{byte_value:02x}",
                             stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
        
        clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



# Additional LA for S1[0] and S1[12]

In [ ]:
equal_prefix = "traces_equal"
not_equal_prefix = "traces_not_equal"

## Specific t-test for STM - Key 1

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-STM"
output_folder = "_LA-STM-1-ADDITIONAL-S1-WORDS"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for word_pos in [0,12]:
    for byte_pos in chain(range(4)):
        for byte_value in range(0x100):
            # Print the current time
            print(f"State [{word_pos}] Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
            
            sort_traces_and_split_s1(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                        chunks = 10, word_index=word_pos, byte_index=byte_pos, byte_value=byte_value)
            
            
            sicak.perform_t_test(folder=output_folder, 
                                 trace_len=TRACE_LEN, 
                                 file_count=10,
                                 output_file_name=f"t_stats_{word_pos}_{byte_pos}_{byte_value:02x}.npy", 
                                 file_equal_prefix=f"traces_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 file_not_equal_prefix=f"traces_not_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
            
            clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for STM - Key 2

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-STM-2"
output_folder = "_LA-STM-2-ADDITIONAL-S1-WORDS"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for word_pos in [0,12]:
    for byte_pos in chain(range(4)):
        for byte_value in range(0x100):
            # Print the current time
            print(f"State [{word_pos}] Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
            
            sort_traces_and_split_s1(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                        chunks = 10, word_index=word_pos, byte_index=byte_pos, byte_value=byte_value)
            
            
            sicak.perform_t_test(folder=output_folder, 
                                 trace_len=TRACE_LEN, 
                                 file_count=10,
                                 output_file_name=f"t_stats_{word_pos}_{byte_pos}_{byte_value:02x}.npy", 
                                 file_equal_prefix=f"traces_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 file_not_equal_prefix=f"traces_not_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
            
            clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for STM - Key 1

In [ ]:
traces_folder = "ChaCha-1000-000-Random-Nonce-XMEGA"
output_folder = "_LA-XMEGA-1-ADDITIONAL-S1-WORDS"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for word_pos in [0,12]:
    for byte_pos in chain(range(4)):
        for byte_value in range(0x100):
            # Print the current time
            print(f"State [{word_pos}] Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
            
            sort_traces_and_split_s1(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                        chunks = 10, word_index=word_pos, byte_index=byte_pos, byte_value=byte_value)
            
            
            sicak.perform_t_test(folder=output_folder, 
                                 trace_len=TRACE_LEN, 
                                 file_count=10,
                                 output_file_name=f"t_stats_{word_pos}_{byte_pos}_{byte_value:02x}.npy", 
                                 file_equal_prefix=f"traces_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 file_not_equal_prefix=f"traces_not_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
            
            clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)



## Specific t-test for STM - Key 1

In [ ]:
traces_folder = "ChaCha-100-000-Random-Nonce-XMEGA-2"
output_folder = "_LA-XMEGA-2-ADDITIONAL-S1-WORDS"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
gc.collect()

In [ ]:
CORRECT_KEY = np.fromfile(f"{traces_folder}/key.bin", dtype=np.uint32)

read_info(traces_folder)
read_nonces(traces_folder)
compress_traces_into_one_file(traces_folder)

In [ ]:
S1_ARRAY = calculate_state_s1()

In [ ]:
for word_pos in [0,12]:
    for byte_pos in chain(range(4)):
        for byte_value in range(0x100):
            # Print the current time
            print(f"State [{word_pos}] Byte[{byte_pos}], Value = {hex(byte_value)}, Current time: ", datetime.now())
            
            sort_traces_and_split_s1(output_folder, input_file = f"{traces_folder}\\traces_random_all.bin",
                        chunks = 10, word_index=word_pos, byte_index=byte_pos, byte_value=byte_value)
            
            
            sicak.perform_t_test(folder=output_folder, 
                                 trace_len=TRACE_LEN, 
                                 file_count=10,
                                 output_file_name=f"t_stats_{word_pos}_{byte_pos}_{byte_value:02x}.npy", 
                                 file_equal_prefix=f"traces_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 file_not_equal_prefix=f"traces_not_equal_{word_pos:02x}_{byte_pos:02x}_{byte_value:02x}",
                                 stan_exe_path = r".\..\sicak-1.2.1\stan.exe")
            
            clean_up_trace_files(output_folder, equal_prefix, not_equal_prefix)

